In [1]:
##Coursera Capstone
##IBM Applied data Science capstone 
##Opening Thai Restaurant in New York City USA.


In [2]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium 
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [3]:
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
##1.	New York data Containing the neighborhoods and boroughs 

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [11]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [12]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'LRWUZSVINXLUPBYFSISP1RWEGCZNL0XYVISC1ZMJG5AG5VFB'
CLIENT_SECRET = 'GFMHTMHGMUNA4XNFXD5AP1LB20VMSHWN040L0PTOYCVLBU0N'
VERSION = '20210409'

In [13]:
##Thairestauant in neighoorhood 

In [14]:
#https://developer.foursquare.com/docs/resources/categories
#ThaiRestaurant=4bf58dd8d48988d149941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_ThaiRestaurant = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d149941735')
newyork_venues_ThaiRestaurant.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Siam Square,40.878796,-73.916701,Thai Restaurant
1,Marble Hill,40.876551,-73.910660,Nam Thai,40.886388,-73.910025,Thai Restaurant
2,Marble Hill,40.876551,-73.910660,Moon Thai Kitchen,40.886963,-73.904870,Thai Restaurant
3,Chinatown,40.715618,-73.994279,Noree Thai Bazaar,40.717900,-73.992966,Thai Restaurant
4,Chinatown,40.715618,-73.994279,Wayla,40.718291,-73.992584,Thai Restaurant


In [15]:
newyork_venues_ThaiRestaurant.shape

(951, 7)

In [16]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [17]:

map_newyork_ThaiRestaurant = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_ThaiRestaurant, 'blue', map_newyork_ThaiRestaurant)

map_newyork_ThaiRestaurant

In [18]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
manhattan_grouped = newyork_venues_ThaiRestaurant.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_ThaiRestaurant['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,9,9,9,9,9,9
Carnegie Hill,25,25,25,25,25,25
Central Harlem,4,4,4,4,4,4
Chelsea,30,30,30,30,30,30
Chinatown,35,35,35,35,35,35
Civic Center,25,25,25,25,25,25
Clinton,43,43,43,43,43,43
East Harlem,11,11,11,11,11,11
East Village,47,47,47,47,47,47


In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_ThaiRestaurant[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_ThaiRestaurant['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Chinese Restaurant,Filipino Restaurant,Food Truck,Indian Restaurant,Japanese Restaurant,Malay Restaurant,Ramen Restaurant,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Chinese Restaurant,Filipino Restaurant,Food Truck,Indian Restaurant,Japanese Restaurant,Malay Restaurant,Ramen Restaurant,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Battery Park City,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.888889,0.000000,0.000000,0.000000
1,Carnegie Hill,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.866667,0.000000,0.000000,0.000000
4,Chinatown,0.085714,0.000000,0.028571,0.000000,0.000000,0.000000,0.028571,0.000000,0.028571,0.742857,0.028571,0.057143,0.000000
5,Civic Center,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.840000,0.000000,0.040000,0.000000
6,Clinton,0.023256,0.023256,0.000000,0.046512,0.000000,0.000000,0.000000,0.046512,0.000000,0.860465,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
8,East Village,0.042553,0.042553,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.787234,0.021277,0.021277,0.021277
9,Financial District,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900000,0.000000,0.000000,0.000000


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Thai Restaurant,Food Truck,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
1,Carnegie Hill,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
2,Central Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
3,Chelsea,Thai Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant
4,Chinatown,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant


In [24]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 3, 2, 3, 3, 1, 2, 3], dtype=int32)

In [25]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
1,Manhattan,Chinatown,40.715618,-73.994279,2,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
3,Manhattan,Inwood,40.867684,-73.921210,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Thai Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
16,Murray Hill,Thai Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
34,Sutton Place,Thai Restaurant,Food Truck,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
35,Turtle Bay,Thai Restaurant,Food Truck,Asian Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
2,Washington Heights,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
3,Inwood,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
4,Hamilton Heights,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
5,Manhattanville,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
6,Central Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
7,East Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
10,Lenox Hill,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
11,Roosevelt Island,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
24,West Village,Thai Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant


In [29]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant
18,Greenwich Village,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Chinese Restaurant,Wine Bar,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
19,East Village,Thai Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Filipino Restaurant,Ramen Restaurant,Malay Restaurant
20,Lower East Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Filipino Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
22,Little Italy,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant
23,Soho,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Wine Bar,Vegetarian / Vegan Restaurant,Ramen Restaurant,Japanese Restaurant,Indian Restaurant
31,Noho,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Wine Bar,Vegetarian / Vegan Restaurant,Filipino Restaurant,Chinese Restaurant,Ramen Restaurant,Malay Restaurant


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
Out[26]


In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
9,Yorkville,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
12,Upper West Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
21,Tribeca,Thai Restaurant,Vietnamese Restaurant,Asian Restaurant,Wine Bar,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
25,Manhattan Valley,Thai Restaurant,Indian Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant
26,Morningside Heights,Thai Restaurant,Indian Restaurant,Wine Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck


In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
Out[26]